In [4]:
import pandas as pd
import os
import pickle
import glob 
from typing import *

data_folder = 'data/raw_data/'

datafolder = os.path.expanduser("~/PycharmProjects/anespo-DWWP.4")

print(os.listdir())
try:
    import sys
    sys.path.append('./py_scripts/')
    from py_functions.pandas_utilities import read_csv_files, check_dataframes, trim_dataframes, create_reports
    print("Successfully imported functions and data.")
except:
    print("Core folder not found")
    exit()

['.git', '141.ipynb', 'data', 'py_functions', 'data_cleaning.ipynb', '.gitignore', 'html_reports', 'requirements.txt', 'covid_infections_map.html', 'covid_dashboard.html', 'README.md', 'data_visualization.ipynb', 'ydata_profiling_reports', '.vscode']
Successfully imported functions and data.


### Reading CSV Files

In [5]:
csv_files = glob.glob(os.path.join(data_folder, "*.csv"))
dataframes = read_csv_files(csv_files)

File acquired:
Case.csv

File acquired:
PatientInfo.csv

File acquired:
Policy.csv

File acquired:
Region.csv

File acquired:
Time.csv

File acquired:
TimeAge.csv

File acquired:
TimeGender.csv

File acquired:
TimeProvince.csv



### Checking DataFrames

Using custom function that checks dataframes integrity and remove possible errors within columns and rows with pandas utilities.

In [6]:
trim_dataframes(dataframes)
check_dataframes(dataframes)

Checking dataframe: case_df
0 duplicate values detected for case_df
0 NA values detected for case_df

Checking dataframe: patientinfo_df
0 duplicate values detected for patientinfo_df
NAs:

patient_id               0
sex                   1122
age                   1380
country                  0
province                 0
city                    94
infection_case         919
infected_by           3819
contact_number        4374
symptom_onset_date    4475
confirmed_date           3
released_date         3578
deceased_date         5099
state                    0
dtype: int64


Checking dataframe: policy_df
0 duplicate values detected for policy_df
NAs:

policy_id      0
country        0
type           0
gov_policy     0
detail         2
start_date     0
end_date      37
dtype: int64


Checking dataframe: region_df
0 duplicate values detected for region_df
0 NA values detected for region_df

Checking dataframe: time_df
0 duplicate values detected for time_df
0 NA values detected for time

It would be a good idea to go on the website once more, and check the dataframes to understand the nature of these missing and duplicated values. In such cases, it is good to remember that having missing values could be a positive sign. For example, for a table such as the 'patient_id', where we have a column named 'deceased_date', we do expect missing values as not all infected people have (luckily) died from the disease.

Whereas for duplicates we don't seem to find any at all, and this is a great sign which means each row has been systematically positioned so as to avoid duplicates and enhance the quality of the data with more clarity. To get an even better, and more in-depth understanding of our data, let's use the library 'ydata_profiling', that can give us extremely detailed analysis of our dataframes. This takes some time, so I will leave it commented out.

In [4]:
create_reports(dataframes, data_folder)

Checking the data reports from ydata_profiling we can see all the detailed information about our data, and that everything is fine. This dataset is exceptionally clean and ready to be analysed, we will just do a few transformations.

We will keep the 'policy_df' and 'region_df' tables as registry tables we can pull more relevant information from.

Now, to improve easiness of accessing data, we can save the most important dataframe to specific variables.



In [5]:
patients = dataframes['patientinfo_df']

## Data Cleaning

We can now start with approaching the data, to understand and elaborate it. We should proceed within each dataframe, one by one, and see what we can fix or improve.

### patientinfo_df

We saw earlier there were some NAs and null values in the 'patientinfo_df', and I want to reiterate that it is not a problem as they are just missing values from specifics such as deceased or confirmed cases. We could instead impute missing values for columns such as 'gender' by taking the ratio of male/female in the dataset and applying it to the dataset, but this is not a good approach as sometimes the person might want to remain unbalanced in their gender description and that is fine. We will just take this possibility into account in the analysis.

There are only a few things we could do, without risking to impact the data quality too much:

- we can create a numeric column for the age, as we don't have one in the 'patientinfo_df' currently. 
- we can remove the NAs in the age as they will not be useful in the analysis.
- make the date columns an effective datetime type.
- we should make it more understandable whether the underlying dates are all setup properly (no deceased date also means no deceased state etc)

In [6]:
# Drop rows with NaN values in the 'age' column of the 'patientinfo_df'
patients.dropna(subset=['age'], inplace=True)
patients.loc[:, 'age_numeric'] = patients['age'].str[:-1].astype(int)

# Transform columns to datetime type
date_cols = ["confirmed_date", "released_date", "deceased_date"]

for col in date_cols:
    patients[col] = pd.to_datetime(patients[col])

# Prepare a column that defines the state by gender
# patients.loc[:, "state_by_gender"] = patients["state"] + "_" + patients["sex"]

Now we need to work with the dates, and as such it might be interesting to obtain the time since the confirmed date, the time since the released date and the time since the deceased date. 

In [7]:
# duration of infection:
patients["time_to_release_since_confirmed"] = patients["released_date"] - patients["confirmed_date"]
patients["time_to_death_since_confirmed"] = patients["deceased_date"] - patients["confirmed_date"]
patients["duration_since_confirmed"] = patients[["time_to_release_since_confirmed", "time_to_death_since_confirmed"]].min(axis=1)

In [8]:
# imputing values for average release time
average_release_time = patients["time_to_release_since_confirmed"].mean()
released_with_no_date = patients[(patients['state'] == 'released') & (patients['released_date'].isnull())].shape[0]

patients.loc[patients['released_date'].isnull(), 'released_date'] = patients['confirmed_date'] + pd.to_timedelta(average_release_time, unit='D')

# Remove remaining NaT values
patients.dropna(subset=['released_date'], inplace=True)
released_with_no_date = patients[(patients['state'] == 'released') & (patients['released_date'].isnull())]
deceased_with_no_date = patients[(patients['state'] == 'deceased') & (patients['deceased_date'].isnull())]

print(released_with_no_date.shape[0], 'released with no release date')
print(deceased_with_no_date.shape[0], 'deceased with no deceased date')

patients

0 released with no release date
9 deceased with no deceased date


,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state,age_numeric,state_by_gender,time_to_release_since_confirmed,time_to_death_since_confirmed,duration_since_confirmed
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaT,released,50,released_male,13 days,NaT,13 days
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaT,released,30,released_male,32 days,NaT,32 days
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaT,released,50,released_male,20 days,NaT,20 days
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaT,released,20,released_male,16 days,NaT,16 days
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaT,released,20,released_female,24 days,NaT,24 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5156,7000000011,male,30s,Korea,Jeju-do,Jeju-do,contact with patient,7000000009,5,NaN,2020-04-03,2020-05-19,NaT,released,30,released_male,46 days,NaT,46 days
5157,7000000012,female,20s,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,9,NaN,2020-04-03,2020-05-05,NaT,released,20,released_female,32 days,NaT,32 days
5158,7000000013,female,10s,China,Jeju-do,Jeju-do,overseas inflow,NaN,6,NaN,2020-04-14,2020-04-26,NaT,released,10,released_female,12 days,NaT,12 days
5159,7000000014,female,30s,Korea,Jeju-do,Jeju-do,Itaewon Clubs,NaN,140,NaN,2020-05-09,2020-06-12,NaT,released,30,released_female,34 days,NaT,34 days


In [12]:
deceased_mask = (patients['state'] == 'deceased') & (patients['deceased_date'].isnull())
patients.loc[deceased_mask, 'state'] = 'released'

# for case fetality rate:
patients["state_deceased"] = (patients["state"] == "deceased").astype("int8")

patients

,patient_id,sex,age,country,province,city,infection_case,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state,age_numeric,state_by_gender,time_to_release_since_confirmed,time_to_death_since_confirmed,duration_since_confirmed,state_deceased
0,1000000001,male,50s,Korea,Seoul,Gangseo-gu,overseas inflow,NaN,75,2020-01-22,2020-01-23,2020-02-05,NaT,released,50,released_male,13 days,NaT,13 days,0
1,1000000002,male,30s,Korea,Seoul,Jungnang-gu,overseas inflow,NaN,31,NaN,2020-01-30,2020-03-02,NaT,released,30,released_male,32 days,NaT,32 days,0
2,1000000003,male,50s,Korea,Seoul,Jongno-gu,contact with patient,2002000001,17,NaN,2020-01-30,2020-02-19,NaT,released,50,released_male,20 days,NaT,20 days,0
3,1000000004,male,20s,Korea,Seoul,Mapo-gu,overseas inflow,NaN,9,2020-01-26,2020-01-30,2020-02-15,NaT,released,20,released_male,16 days,NaT,16 days,0
4,1000000005,female,20s,Korea,Seoul,Seongbuk-gu,contact with patient,1000000002,2,NaN,2020-01-31,2020-02-24,NaT,released,20,released_female,24 days,NaT,24 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5156,7000000011,male,30s,Korea,Jeju-do,Jeju-do,contact with patient,7000000009,5,NaN,2020-04-03,2020-05-19,NaT,released,30,released_male,46 days,NaT,46 days,0
5157,7000000012,female,20s,Korea,Jeju-do,Jeju-do,overseas inflow,NaN,9,NaN,2020-04-03,2020-05-05,NaT,released,20,released_female,32 days,NaT,32 days,0
5158,7000000013,female,10s,China,Jeju-do,Jeju-do,overseas inflow,NaN,6,NaN,2020-04-14,2020-04-26,NaT,released,10,released_female,12 days,NaT,12 days,0
5159,7000000014,female,30s,Korea,Jeju-do,Jeju-do,Itaewon Clubs,NaN,140,NaN,2020-05-09,2020-06-12,NaT,released,30,released_female,34 days,NaT,34 days,0


Another thing we could do is to impute the city in which the patient has been found to have been infected, but having the 'province' column as well, we risk to impute the wrong information or we have to make such a big effort that it would not be worth it. For the other dataframes with null values, we wil apply the same concept as before by leaving them as they are as it would not make sense to impute the weather or to remove the policies with no 'end_date' as they might just be still active.


Therefore, I think the data is ready to be analyzed now, save our dataframes file and we can go over to the next notebook, 'data_analysis'.

In [13]:
dataframes['patientinfo_df'] = patients

# Save dataframes dictionary to a file
with open('dataframes.pickle', 'wb') as file:
    pickle.dump(dataframes, file)